Jupyter notebook één team.
In the code below, all needed imports for the following code are made up here.

In [76]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import numpy as np
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols

#Imports used for python tf-idf
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

import sqlalchemy


In the code below, the option to display graphs inline will be set to true.

In [30]:
%config IPCompleter.greedy = True
%matplotlib inline

Reading the data and assign it to the correct variable.

In [77]:
parse_dates = ['Date received', 'Date sent to company']
dtypes = {'Date received': str,
          'Product': "category",
          'Sub-product': "category",
          'Issue': "category",
          'Sub-issue':"category",
          'Consumer complaint narrative':str,
          'Company public response':str,
          'Company':"category",
          'State':"category",
          'ZIP code':str,
          'Tags':"category",
          'Consumer consent provided?':str,
          'Submitted via':"category",
          'Date sent to company':str,
          'Company response to consumer':str,
          'Timely response?':str,
          'Consumer disputed?':str,
          'Complaint ID':int}

DS1_data = pd.read_csv("../Data/complaints-2023-04-25_05_07.csv", low_memory = False, dtype = dtypes, parse_dates = parse_dates)
DS1_data[['Timely response?','Consumer disputed?']] = DS1_data[['Consumer disputed?','Timely response?']].replace({'Yes': True, 'No':False}).astype(bool)
DS1_data['Consumer consent provided?'] = DS1_data['Consumer consent provided?'].replace({'Consent provided': True, '':False}).astype(bool)
DS1_data = DS1_data[pd.notnull(DS1_data['Consumer complaint narrative'])]   #Drops the missing rows with no  complaint.
DS1_data.drop(columns=['Sub-issue'])

C:\Users\SKIKK\AppData\Local\Temp\ipykernel_53668\1843986035.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  DS1_data = pd.read_csv("../Data/complaints-2023-04-25_05_07.csv", low_memory = False, dtype = dtypes, parse_dates = parse_dates)
C:\Users\SKIKK\AppData\Local\Temp\ipykernel_53668\1843986035.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  DS1_data = pd.read_csv("../Data/complaints-2023-04-25_05_07.csv", low_memory = False, dtype = dtypes, parse_dates = parse_dates)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2015-07-23,Mortgage,FHA mortgage,"Loan modification,collection,foreclosure",NaN,""" The California Homeowner Bill of Rights beca...",Company chooses not to provide a public response,"CITIBANK, N.A.",CA,95965,NaN,True,Web,2015-07-30,Closed with explanation,True,True,1484039
1,2021-12-09,Mortgage,FHA mortgage,Closing on a mortgage,NaN,"Hello, I am trying to refinance my property an...",Company has responded to the consumer and the ...,"LoanCare, LLC",VA,20136,NaN,True,Web,2021-12-09,Closed with explanation,True,True,4991921
2,2016-09-29,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,Bank of America has caused to be recorded NOD ...,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",CA,92114,NaN,True,Web,2016-09-29,Closed with explanation,False,True,2138230
4,2021-12-09,Mortgage,Conventional home mortgage,Struggling to pay mortgage,NaN,We have 2 loans w/ Loan CarXXXX XXXX. We were ...,Company has responded to the consumer and the ...,"LoanCare, LLC",IL,60471,Servicemember,True,Web,2021-12-09,Closed with explanation,True,True,4993695
5,2016-01-05,Mortgage,Home equity loan or line of credit,"Loan servicing, payments, escrow account",NaN,My husband has died and I am filing bankruptcy...,NaN,"CITIBANK, N.A.",VT,05664,Older American,True,Web,2016-01-05,Closed with explanation,False,True,1729048
6,2022-03-01,Mortgage,Conventional home mortgage,Applying for a mortgage or refinancing an exis...,NaN,I am writing this complaint in hopes the CFPB ...,NaN,Ocwen Financial Corporation,CT,06239,Older American,True,Web,2022-03-01,Closed with explanation,True,True,5273876
7,2021-10-21,Mortgage,FHA mortgage,Struggling to pay mortgage,NaN,In or about XXXX 2015 I had lost my job and wa...,Company has responded to the consumer and the ...,"LAKEVIEW LOAN SERVICING, LLC",NJ,07206,Servicemember,True,Web,2021-10-21,Closed with explanation,True,True,4831612
8,2022-02-14,Mortgage,Conventional home mortgage,Trouble during payment process,NaN,Mortgage company : Community Loan Servicing 1....,NaN,"Community Loan Servicing, LLC (formerly known ...",CT,06037,NaN,True,Web,2022-02-14,Closed with explanation,True,True,5219315
9,2016-12-01,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,I began a refinance with Citizens Bank. I paid...,NaN,"CITIZENS FINANCIAL GROUP, INC.",NJ,08107,NaN,True,Web,2016-12-01,Closed with explanation,True,True,2232457
10,2016-08-02,Mortgage,FHA mortgage,Settlement process and costs,NaN,Complaint Description : Nationstar is basicall...,NaN,NATIONSTAR MORTGAGE LLC,GA,30188,NaN,True,Web,2016-08-02,Closed with explanation,False,True,2042253


In [ ]:
db = sqlalchemy.create_engine('sqlite:///StaterData.db')
DS1_data.to_sql("mortgage complaints",db,if_exists="replace")